In [2]:
import weaviate
import weaviate
import weaviate.classes as wvc
import os
import requests
import json

# connect to local instance (docker compose) of weaviate
client = weaviate.connect_to_local()

In [77]:
client.collections.delete('Question')
# fill the collection
resp = requests.get('https://raw.githubusercontent.com/weaviate-tutorials/quickstart/main/data/jeopardy_tiny.json')
data = json.loads(resp.text)  # Load data

question_objs = list()
for i, d in enumerate(data):
    question_objs.append({
        "answer": d["Answer"],
        "question": d["Question"],
        "category": d["Category"],
    })

questions = client.collections.get("Question")
questions.data.insert_many(question_objs)




BatchObjectReturn(all_responses=[UUID('c80b9336-40d8-4647-a88d-1f8cc1074ce0'), UUID('0ce2a324-92b9-4214-9e0b-d7860b5c06ba'), UUID('9ee93515-111c-4371-9788-a519a98f73b5'), UUID('0a3c1351-caa0-4856-aea5-ea36c50fa2be'), UUID('6714892c-705d-4ecd-9fd2-7ea823b9833e'), UUID('1978932c-8e18-436a-b048-88fe73288d37'), UUID('25e31dc1-86f8-46a7-8a18-a546088af34a'), UUID('9a07d19b-281e-4b0c-9d19-ceb5f9fbb398'), UUID('6983d7bb-f474-45dd-9332-ecfe800b861c'), UUID('90eef5d6-7b8a-4681-95dc-283fed7becdc')], elapsed_seconds=2.12142276763916, errors={}, uuids={0: UUID('c80b9336-40d8-4647-a88d-1f8cc1074ce0'), 1: UUID('0ce2a324-92b9-4214-9e0b-d7860b5c06ba'), 2: UUID('9ee93515-111c-4371-9788-a519a98f73b5'), 3: UUID('0a3c1351-caa0-4856-aea5-ea36c50fa2be'), 4: UUID('6714892c-705d-4ecd-9fd2-7ea823b9833e'), 5: UUID('1978932c-8e18-436a-b048-88fe73288d37'), 6: UUID('25e31dc1-86f8-46a7-8a18-a546088af34a'), 7: UUID('9a07d19b-281e-4b0c-9d19-ceb5f9fbb398'), 8: UUID('6983d7bb-f474-45dd-9332-ecfe800b861c'), 9: UUID('90ee

In [78]:
# dense vector (single vector representing whole text) search 
questions = client.collections.get("Question")
response = questions.query.near_text(
    query="tropospheric",
    limit=2,
    include_vector=True,
    return_metadata=wvc.query.MetadataQuery(distance=True)
)
print(response.objects[0].vector)  # Inspect the first object
print(response.objects[0])


{'default': [-0.025153737515211105, 0.0017846764530986547, 0.023027831688523293, 0.04007139429450035, -0.04249945655465126, -0.013377227820456028, 0.020174214616417885, -0.016248684376478195, 0.035785745829343796, -0.03398603945970535, -0.030122708529233932, -0.0018783682025969028, 0.004990114364773035, 0.00029949643067084253, 0.027021901682019234, 0.01486821286380291, 0.0027144348714500666, 0.014392356388270855, 0.005287353880703449, 0.011655553244054317, 0.05324673280119896, 0.014734592288732529, -0.02183738350868225, -0.05737548694014549, -0.07883071154356003, 0.023165235295891762, 0.043296728283166885, -0.010053180158138275, 0.034139957278966904, 0.005506639368832111, -0.013611456379294395, 0.0298339631408453, -0.0055149877443909645, -0.028885694220662117, -0.01193637028336525, 0.0573740117251873, 0.04103652015328407, 0.022826645523309708, -0.005913592875003815, 0.007898261770606041, 0.012359126470983028, -0.011324121616780758, -0.06492403894662857, -0.019880501553416252, -0.022900

In [79]:
# bm25 ~= keyword search
response = questions.query.bm25(
    query="tropospheric",
    return_metadata=wvc.query.MetadataQuery(score=True),
    limit=1
)

for o in response.objects:
    print(o.properties)
    print(o.metadata.score)



{'answer': 'the atmosphere', 'question': 'Changes in the tropospheric layer of this are what gives us weather', 'category': 'SCIENCE'}
0.5992271304130554


In [1]:
# hybrid search = dense vector search + bm25
response = questions.query.hybrid(
    query="oves excess glucose from the blood & stores it as g",
    limit=3
)

for o in response.objects:
    print(o.properties)


NameError: name 'questions' is not defined

In [5]:
client = weaviate.Client("http://localhost:8080")

# Fetch the first item from the "Question" collection
result = client.data_object.get(class_name="Caption", limit=1)

# Print the first item
if result and 'objects' in result and len(result['objects']) > 0:
    first_item = result['objects'][0]
    print(first_item)
else:
    print("No items found in the 'Question' collection.")


/home/erwan/anaconda3/envs/scienceinfuse/lib/python3.9/site-packages/weaviate/warnings.py:162: DeprecationWarning: Dep016: Python client v3 `weaviate.Client(...)` connections and methods are deprecated. Update
            your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.

            For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
            For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration
            
  warnings.warn(


No items found in the 'Question' collection.


In [ ]:
questions.iterator()